In [18]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
gdp_df = pd.read_csv("./data/gdp/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4901850.csv")
pop_df = pd.read_csv("./data/population/API_SP.POP.TOTL_DS2_en_csv_v2_5358404.csv")
spi_df = pd.read_csv("./data/SPI_index_mean_and_centroid_latitude.csv")

In [3]:
country_codes = list(spi_df["iso3c"])

### Filtering GDP and population data for the countries for which SPI and centroid data exists

In [4]:
gdp_df = gdp_df[gdp_df["Country Code"].isin(country_codes)].reset_index(drop=True)
pop_df = pop_df[pop_df["Country Code"].isin(country_codes)].reset_index(drop=True)

### Checking for missing values in GDP and population data

In [5]:
start_year = 1991
end_year = 2021
columns_to_check = [str(year) for year in range(start_year, end_year+1)]
columns_to_check.insert(0, "Country Name")

In [6]:
def check_missing_data(df, start_year, end_year):
    countries_with_missing_data = dict()

    for i, row in df[columns_to_check].iterrows():
        missing_data_years = list(row[row.isna()].index)
        if len(missing_data_years) != 0:
            countries_with_missing_data[row["Country Name"]] = missing_data_years

    return countries_with_missing_data

In [7]:
countries_missing_gdp_data = check_missing_data(gdp_df[columns_to_check], start_year, end_year)
countries_missing_pop_data = check_missing_data(pop_df[columns_to_check], start_year, end_year)

### Dropping countries for which the GDP data is missing for >=50% of the period of interest(1991-2021). Including 1991 because we need this to be able to interpolate monthly GDP for the year 1992 

In [8]:
countries_to_drop = []
threshold = 2021-1991+1
for country, data in countries_missing_gdp_data.items():
    if len(data) >= threshold//2:
        countries_to_drop.append(country)

print("Countries with more than 50% missing GDP data: ", countries_to_drop)

gdp_df = gdp_df[~gdp_df["Country Name"].isin(countries_to_drop)]
gdp_df = gdp_df.reset_index(drop=True)

pop_df = pop_df[~pop_df["Country Name"].isin(countries_to_drop)]
pop_df = pop_df.reset_index(drop=True)

Countries with more than 50% missing GDP data:  ['Somalia', 'South Sudan']


### Dropping unneccessary columns

In [9]:
columns_to_drop = [str(year) for year in range(1960, 1991)]
columns_to_drop.append("Indicator Code")
gdp_df.drop(axis=1, columns=columns_to_drop, inplace=True)
pop_df.drop(axis=1, columns=columns_to_drop, inplace=True)

### Evaluating GDP per capita before imputation

In [10]:
gdp_df = gdp_df.sort_values(by="Country Code")
gdp_df = gdp_df.reset_index(drop=True)

pop_df = pop_df.sort_values(by="Country Code")
pop_df = pop_df.reset_index(drop=True)

## checking whether the sorted order of countries is same in both the dfs
assert list(gdp_df["Country Code"]) == list(pop_df["Country Code"]), "Sorted order different"

In [11]:
columns_to_check_copy = columns_to_check.copy()
del columns_to_check_copy[0]

gdp_array = np.array(gdp_df[columns_to_check_copy])
pop_array = np.array(pop_df[columns_to_check_copy])

gdp_per_capita_df = pd.DataFrame(np.divide(gdp_array, pop_array), columns=columns_to_check_copy)
gdp_per_capita_df["Country Name"] = gdp_df["Country Name"]
gdp_per_capita_df["Country Code"] = gdp_df["Country Code"]

new_col_order = list(gdp_per_capita_df.columns[-2:]) + list(gdp_per_capita_df.columns[0:-2])
gdp_per_capita_df = gdp_per_capita_df[new_col_order]

### GDP data imputation using KNN

In [12]:
imputer = KNNImputer(n_neighbors=5, weights="distance")

In [13]:
columns_to_impute = list(gdp_per_capita_df.columns[2:])
gdp_per_capita_df[columns_to_impute] = imputer.fit_transform(gdp_per_capita_df[columns_to_impute])

In [15]:
gdp_per_capita_df.to_csv("./data/gdp/gdp_per_capita.csv",index=False)